# Preprocessing (up to and including split)


## Loading dataset

In [1]:
import pandas as pd

#INPUT AND OUTPUT LOCATION FOLDER 
location = '/Users/klaaskorteweg/Desktop/Master Thesis/'

#loading the GTD (orginal and unprocessed)
gtd = 'globalterrorismdb_0522dist.xlsx'

raw_df = pd.read_excel(location+gtd)

## Data Filtering and Data Cleaning

### Data Filtering: Dropping non relevant variables
Only selecting the meaningful and (potentially) relevant variables after extensive analysis based on the codebook.

In [2]:
#dropping all non relevant variables, leaving only the selected variables: 

selected_variables = ["iyear", "imonth", "iday", "country_txt", "region_txt", "provstate", "city",
    "attacktype1_txt", "targtype1_txt", "gname", "gsubname", "weaptype1_txt", "weapsubtype1_txt","crit1", "crit2", "crit3", "multiple", "suicide", "individual",
    "weaptype2_txt", "weapsubtype2_txt", "weaptype3_txt", "weapsubtype3_txt", "weaptype4_txt", "weapsubtype4_txt", "INT_LOG",
    "INT_IDEO", "INT_MISC", "INT_ANY"]

df = raw_df[selected_variables].copy()

### Data Cleaning: Checking and Removing Duplicates 
Check duplicates in raw dataset, not in selected df because it's already missing some variables that indicate different events. 



In [3]:
check_vars = raw_df.columns.tolist() #all orginal variables.
duplicates = raw_df[raw_df.duplicated(subset=check_vars,keep=False)]

if duplicates.empty:
    print("No duplicates found based on the specific variables.")
else:
    print("Duplicate rows based on the specified variables (",((duplicates.shape[0])/df.shape[0])*100,"% of dataset):")

No duplicates found based on the specific variables.


### Data Cleaning: Homogenize missing values
Missing values are indicated differently across the data (NaN, "Unknown", 0, -9)

In [4]:
#Missing values information:
def missing_values_table(dataset):
    missing_df = pd.DataFrame(columns=['variable','unique_values','%_nan','%_Unknown','%_0','%_-9','%_total'])
    vars = dataset.columns.tolist()
    for var in vars:
        perc_total = 0

        #unique_values 
        unique_values = dataset[var].nunique()

        #%_nan
        missing_values_count = dataset[var].isnull().sum()
        missing_values_perc = ((missing_values_count/dataset.shape[0])*100)
        perc_total += missing_values_perc

        #%_Unknown
        count_unknown = (dataset[var] == 'Unknown').sum()
        perc_unknown = ((count_unknown/dataset.shape[0])*100)
        perc_total += perc_unknown
        
        #%_0 (in some categories 0 counts as missing, like imonth, iday)
        count_0 = (dataset[var] == 0).sum()
        perc_0 = ((count_0/dataset.shape[0])*100)
        perc_total += perc_0

        #_-9 (in some categories -9 counts as missing, mostly in binary variables)
        count_min9 = (dataset[var] == -9).sum()
        perc_min9 = ((count_min9/dataset.shape[0])*100)
        perc_total += perc_min9


        missing_df.loc[len(missing_df)] = {'variable':var,'unique_values':unique_values,'%_nan':round(missing_values_perc,2),'%_Unknown':round(perc_unknown,2),'%_0':round(perc_0,2),'%_-9':round(perc_min9,2),'%_total':round(perc_total,2)}
    return missing_df

def variable_info(dataset,var,val):
    #unique values for certain variable: 
    print('Number of unique values:',dataset[var].nunique())
    print('Unique values:',dataset[var].unique())
    print()
    #missing value count:
    missing_values_count = dataset[var].isnull().sum()
    print("Number of missing values:",missing_values_count)
    print("Percentage of missing values:",(missing_values_count/dataset.shape[0])*100)
    print()
    #count specific value in variable:
    count_value = (dataset[var] == val).sum()
    print("Number of",val,'in',var,':',count_value)
    print("Percentage of",val,'in',var,':',(count_value/dataset.shape[0])*100)

missing_values_table(df)


,variable,unique_values,%_nan,%_Unknown,%_0,%_-9,%_total
0,iyear,50,0.00,0.00,0.00,0.00,0.00
1,imonth,13,0.00,0.00,0.01,0.00,0.01
2,iday,32,0.00,0.00,0.42,0.00,0.42
3,country_txt,204,0.00,0.00,0.00,0.00,0.00
4,region_txt,12,0.00,0.00,0.00,0.00,0.00
5,provstate,2623,0.00,2.12,0.00,0.00,2.12
6,city,45147,0.20,5.28,0.00,0.00,5.48
7,attacktype1_txt,9,0.00,5.22,0.00,0.00,5.22
8,targtype1_txt,22,0.00,3.42,0.00,0.00,3.42
9,gname,3725,0.00,43.83,0.00,0.00,43.83


In [5]:
import numpy as np

def replace_value(data,var,old_value,new_value):
    data[var].replace(old_value, new_value, inplace=True)


In [6]:
#DATE VARIABLES: 

#iyear:
#no missing values

#imonth: replace 0 with nan
replace_value(df,'imonth',0,np.nan)

#iday: replace 0 with nan
replace_value(df,'iday',0,np.nan)

#LOCATION VARIABLES: 
#provstate: replace 'Unknown' with nan and lowercase variable 
replace_value(df,'provstate','Unknown',np.nan)
replace_value(df,'provstate','unknown',np.nan)

#city: replace 'Unknown' with nan and lowercase variable
replace_value(df,'city','Unknown',np.nan)
replace_value(df,'city','unknown',np.nan)

#ATTACK VARIABLES: 
#attacktype1_txt: replace 'Unknown' with nan
replace_value(df,'attacktype1_txt','Unknown',np.nan)

#targtype1_txt: replace 'Unknown' with nan
replace_value(df,'targtype1_txt','Unknown',np.nan)

#gname: uncertain
replace_value(df,'gname','Unknown',np.nan)

#gsubname: uncertain
#not necessary

#weaptype1_txt: replace 'Unknown' with nan
replace_value(df,'weaptype1_txt','Unknown',np.nan)

#remaining weaptypes: uncertain. 
#not necessary

#CRITERION VARIABLES: 
#crit1, crit2, crit3 are binary and have no missing values 

#INT VARIABLES: 
replace_value(df,'INT_LOG',-9,np.nan)
replace_value(df,'INT_IDEO',-9,np.nan)
replace_value(df,'INT_MISC',-9,np.nan)
replace_value(df,'INT_ANY',-9,np.nan)

In [7]:
missing_values_table(df)

,variable,unique_values,%_nan,%_Unknown,%_0,%_-9,%_total
0,iyear,50,0.00,0.00,0.00,0.0,0.00
1,imonth,12,0.01,0.00,0.00,0.0,0.01
2,iday,31,0.42,0.00,0.00,0.0,0.42
3,country_txt,204,0.00,0.00,0.00,0.0,0.00
4,region_txt,12,0.00,0.00,0.00,0.0,0.00
5,provstate,2621,2.12,0.00,0.00,0.0,2.12
6,city,45145,5.50,0.00,0.00,0.0,5.50
7,attacktype1_txt,8,5.22,0.00,0.00,0.0,5.22
8,targtype1_txt,21,3.42,0.00,0.00,0.0,3.42
9,gname,3724,43.83,0.00,0.00,0.0,43.83


### Data Cleaning: Cleaning Text Data
Cleaning textual data, like country_txt, provstate and city



In [8]:
#lowercase provstate and city variable:

def lowercase_variable(data,var):
    data[var] = data[var].str.lower()

lowercase_variable(df,'provstate')
lowercase_variable(df,'city')

In [9]:
missing_values_table(df)

,variable,unique_values,%_nan,%_Unknown,%_0,%_-9,%_total
0,iyear,50,0.00,0.00,0.00,0.0,0.00
1,imonth,12,0.01,0.00,0.00,0.0,0.01
2,iday,31,0.42,0.00,0.00,0.0,0.42
3,country_txt,204,0.00,0.00,0.00,0.0,0.00
4,region_txt,12,0.00,0.00,0.00,0.0,0.00
5,provstate,2609,2.12,0.00,0.00,0.0,2.12
6,city,44761,5.50,0.00,0.00,0.0,5.50
7,attacktype1_txt,8,5.22,0.00,0.00,0.0,5.22
8,targtype1_txt,21,3.42,0.00,0.00,0.0,3.42
9,gname,3724,43.83,0.00,0.00,0.0,43.83


## Split

In [10]:
from sklearn.model_selection import train_test_split
import pandas as pd

def split(data,size=0.2,split_method=None):
    X = data.drop(columns=['targtype1_txt'])
    y = data['targtype1_txt']
    if split_method == None:
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=size,random_state=42)
        train_df = pd.concat([X_train,y_train],axis=1)
        test_df = pd.concat([X_test,y_test],axis=1)
    elif split_method == 'stratified':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, stratify=y, random_state=42)
        train_df = pd.concat([X_train,y_train],axis=1)
        test_df = pd.concat([X_test,y_test],axis=1)
    elif split_method == 'time':
        split_index = int(len(data)*(1-size))
        train_df = data.iloc[:split_index]
        test_df = data.iloc[split_index:]
    return train_df, test_df

train_df, test_df = split(df, size=0.2,split_method = 'time')


In [11]:
train_df

,iyear,imonth,iday,country_txt,region_txt,provstate,city,attacktype1_txt,targtype1_txt,gname,...,weaptype2_txt,weapsubtype2_txt,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,1970,7.0,2.0,Dominican Republic,Central America & Caribbean,national,santo domingo,Assassination,Private Citizens & Property,MANO-D,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,1970,NaN,NaN,Mexico,North America,federal,mexico city,Hostage Taking (Kidnapping),Government (Diplomatic),23rd of September Communist League,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0
2,1970,1.0,NaN,Philippines,Southeast Asia,tarlac,NaN,Assassination,Journalists & Media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
3,1970,1.0,NaN,Greece,Western Europe,attica,athens,Bombing/Explosion,Government (Diplomatic),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
4,1970,1.0,NaN,Japan,East Asia,fukouka,fukouka,Facility/Infrastructure Attack,Government (Diplomatic),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167759,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167760,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167761,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167762,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0


In [12]:
test_df

,iyear,imonth,iday,country_txt,region_txt,provstate,city,attacktype1_txt,targtype1_txt,gname,...,weaptype2_txt,weapsubtype2_txt,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
167764,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167765,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167766,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167767,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
167768,2016,9.0,21.0,Iraq,Middle East & North Africa,saladin,dawr district,Bombing/Explosion,Private Citizens & Property,Islamic State of Iraq and the Levant (ISIL),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209701,2020,12.0,31.0,Yemen,Middle East & North Africa,al hudaydah,sabaa,Bombing/Explosion,Private Citizens & Property,Houthi extremists (Ansar Allah),...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
209702,2020,12.0,31.0,Yemen,Middle East & North Africa,al hudaydah,beit maghari,Bombing/Explosion,Private Citizens & Property,Houthi extremists (Ansar Allah),...,Explosives,Unknown Explosive Type,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
209703,2020,12.0,31.0,Germany,Western Europe,lower saxony,leipzig,Facility/Infrastructure Attack,Military,Left-wing extremists,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
209704,2020,12.0,31.0,Afghanistan,South Asia,kabul,kabul,Armed Assault,Private Citizens & Property,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


## Saving dataset(s)

In [13]:
def save_data(data, doc_location,doc_title):
    data.to_excel((doc_location+doc_title),index=False)

save_data(train_df,location,'train_data.xlsx')
save_data(test_df,location,'test_data.xlsx') 